<img style="float: left;" src="figures/first_results.png" width="15%"> 

# <font color='Blue'>Basic geothermal model</font>

## <font color='blue'>Introduction</font>

#### In the first case study, we cover the basic structures and procedures to run simulation with DARTS. 
#### DARTS contains a main module engine: 
* <font color='red'>engines</font>  
    
#### This module is programmed in C++ and exposed to python with compiled packages (libraries) named as
* <font color='red'>engines.pyd</font>
  
#### The module engines.pyd provides the run-simulation needed functionalities, like
* reservoir (mesh) initialization
* well settings
* interpolation operation
* jacobian assembly
* linear and nonlinear iteration

#### The module engines.pyd can be found in the folder <font color='red'>'darts'</font>:

In [ ]:
import darts
import os
dirs = os.listdir(os.path.dirname(darts.__file__))
for f in dirs:
   print(f)

## <font color='blue'>The objectives for the first exercise</font>
1. Be familiar with the basic procedures to run a simulation
2. Exercise on several simulation parameters including timestep controls and mesh 

## 1D Model can be divided into following parts:
<img style="float: left;" src="slides/Slide4.PNG" width="60%">

## <font color='blue'>Let's start !</font>
### Step 1. We need to import <font color='red'>engines</font> and nessesary physical properties into the workspace, just like the import of commonly-used modules such as numpy etc.

In [ ]:
'''Import all important packages from DARTS installation'''
from darts.engines import *
from darts.models.darts_model import DartsModel as model
from darts.physics.geothermal.physics import Geothermal
from darts.physics.geothermal.property_container import PropertyContainer
from darts.physics.properties.iapws.iapws_property_vec import _Backward1_T_Ph_vec
from darts.physics.properties.iapws.iapws_property import iapws_total_enthalpy_evalutor, iapws_temperature_evaluator

from darts.engines import redirect_darts_output
redirect_darts_output('basic_model.log')

import numpy as np
import matplotlib.pyplot as plt

### Step 2. We need to specify the time recorder, which could help to make sure about the performance of the simulator in different parts. Timers can be created in hierachical order.

<img style="float: left;" src="slides/Slide6.PNG" width="60%">

In [ ]:
'''Activate main timers for simulation'''
def activate_timer():
    # Call class constructor and Build timer_node object
    timer = timer_node()

    # Call object members; there are 2 types of members:
    ## <1>Function member:
    timer.start()
    ## <2>Data member:
    timer.node["simulation"] = timer_node()
    timer.node["initialization"] = timer_node()

    # Start initialization
    timer.node["initialization"].start()
    
    return timer

### Step 3. Simulation parameters setting. Here we specify the simulation parameters, e.g.:

* timestep strategy (first_ts, max_ts, mult_ts)
* convergence criteria (tolerance of nonlinear iteration and other parameters controlling iterations)

<img style="float: left;" src="slides/Slide7.PNG" width="60%">

In [ ]:
'''Define main parameters for simulation by overwriting default parameters'''

from darts.engines import sim_params
from darts.models.darts_model import DataTS


### Step 4. Mesh initialization
<img style="float: left;" src="slides/Slide8.PNG" width="60%">

<img style="float: left;" src="slides/Slide9.PNG" width="60%">

In [ ]:
'''Mesh initialization for 1D reservoir with constant transmissibility'''
def init_mesh(nb):
    # Create mesh object by calling the class constructor
    mesh = conn_mesh()

    # Create connection list for 1D reservoir 
    block_m = np.arange(nb - 1, dtype=np.int32)
    block_p = block_m + 1

    # Set constant transmissbility
    permeability = 2
    tranD = np.ones(nb - 1) * 1e-3 * nb 
    tran = tranD * permeability

    # Initialize mesh with connection list
    mesh.init(index_vector(block_m), index_vector(block_p),
              value_vector(tran), value_vector(tranD))

    # Complete mesh initialization
    mesh.reverse_and_sort()
    
    return mesh

### Step 5. Define reservoir properties
* connecting numpy arrays to the mesh
* fill porosity, depth, volume, conduction and heat capacity
* imitate boundary conditions with large volumes

<img style="float: left;" src="slides/Slide10.PNG" width="60%">

In [ ]:
'''Define basic properties for the reservoir'''
def define_reservoir(nb, hcap_in, cond_in):
    # Create numpy arrays wrapped around mesh data (no copying)
    volume = np.array(mesh.volume, copy=False)
    porosity = np.array(mesh.poro, copy=False)
    depth = np.array(mesh.depth, copy=False)

    # Thermal properties
    hcap = np.array(mesh.heat_capacity, copy=False)
    cond = np.array(mesh.rock_cond, copy=False)

    # Assign volume, porosity and depth values
    volume.fill(3000 / nb)
    porosity.fill(0.2)
    depth.fill(1000)
   
    # Assign thermal properties
    hcap.fill(hcap_in)
    cond.fill(cond_in)
    
    # Make first and last blocks large (source/sink)
    volume[0] = 1e10
    volume[nb-1] = 1e10    

### Step 6. Boundary and initial conditions
* initial reservoir conditions
* pressure source and sink
* translation from temperature to enthalpy

<img style="float: left;" src="slides/Slide11.PNG" width="60%">

In [ ]:
'''Mimic boundary conditions for the reservoir'''
def define_initial_conditions():
    # Create numpy wrappers for initial solution
    input_distribution = {'pressure': np.ones(nb) * 200.,
                          'temperature': np.ones(nb) * 348,
                          }

    # Assign initial pressure values
    input_distribution['pressure'][0] = 250
    input_distribution['pressure'][nb-1] = 150

    # Assign enthalpy values:
    # first, define initial temperature (Kelvin) ...
    input_distribution['temperature'][0] = 308

    return input_distribution

### Step 7. Physics initialization
* translate temperature range to enthalpy
* initialize physical ranges

<img style="float: left;" src="slides/Slide12.PNG" width="60%">

In [ ]:
# Define function to calculate enthalpy range corresponding to given pressure and temperature ranges
def calc_enthalpy_range(pres, temp):
    min_e = 1e5
    max_e = 0
    for i in range(len(pres)):
        for j in range(len(pres)):
            state = value_vector([pres[i], 0])
            E = iapws_total_enthalpy_evalutor()
            enth = E.evaluate(state, temp[i])
            if min_e > enth:
                min_e = enth
            if max_e < enth:
                max_e = enth
    return min_e, max_e

'''Create physics from predefined properties from DARTS package'''
def define_physics():
    # Define pressure and temperature ranges for the problem
    min_p = 1
    max_p = 351
    min_T = 273.15
    max_T = 500

    # Define amount of points for OBL 
    n_points = 64
    
    # Evaluate enthalpy range
    min_e, max_e  = calc_enthalpy_range([min_p, max_p], [min_T, max_T]) 

    # Build physics class; the format of constructor:
    # Geothermal(timer, n_points, min_pres, max_pres, min_enth, max_enth)
    property_container = PropertyContainer()
    physics = Geothermal(timer, n_points, min_p, max_p, min_e, max_e)
    physics.add_property_region(property_container)
    physics.init_physics()

    return physics

### Step 8. Engine initialization and run
<img style="float: left;" src="slides/Slide13.PNG" width="60%">

In [ ]:
# Define function to convert enthalpy to temperature
def enthalpy_to_temperature(pres, enth):
    state = value_vector([0, 0])
    temp = np.zeros(len(pres))
    T = iapws_temperature_evaluator()
    for i in range(len(pres)):
        state[0] = pres[i]
        state[1] = enth[i]
        temp[i] = T.evaluate(state)
    return temp

# Define function to plot data profiles
%matplotlib inline
def plot_profile(data, name, sp, ax):
    n = len(data)    
    ax.plot(np.arange(n), data[0:n], 'o')
    ax.set_xlabel('Grid index')
    ax.set_ylabel('%s' % (name))

In [ ]:
# create all model parameters
nb = 30
timer = activate_timer()

mesh = init_mesh(nb)
define_reservoir(nb, hcap_in=2200, cond_in=200)

physics = define_physics()
input_distribution = define_initial_conditions()
physics.set_initial_conditions_from_array(mesh, input_distribution)

m = model()

# Initialize engine
physics.engine.init(mesh, ms_well_vector(),
                    op_vector([physics.acc_flux_itor[0]]),
                    m.params, 
                    timer.node["simulation"])

m.physics = physics
m.platform = 'cpu'
m.has_dfm_well = False

m.set_sim_params(first_ts=0.01, mult_ts=4, max_ts=1)

# Stop initialization timer
timer.node["initialization"].stop()

# Run simulator for 2000 days
m.run_simple(physics=physics, days=2000, data_ts=m.data_ts)

# Print timers (note where most of the time was spent!)
print(timer.print("", ""))

### Step 9. Data processing and plots

<img style="float: left;" src="slides/Slide14.PNG" width="60%">

In [ ]:
# Get numpy wrapper for final solution
X = np.array(physics.engine.X, copy=False)

# Prepare for plotting
fig = plt.figure()   

nc = 2

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
# Plot pressure profile
plot_profile(X[0:nc*nb:nc],'Pressure, bar', 1, axes[0])
# Convert enthalpy to temperature first before plotting
temp = enthalpy_to_temperature(X[0::2], X[1::2])
# Plot temperature profile
plot_profile(temp[0:nb]-273.15,'Temperature, C',2,axes[1])
plt.show()

## <font color='Blue'>Reference solution:</font>
<img style="float: left;" src="figures/first_results.png" width="80%">

## <font color='Blue'>Tasks in this workshop</font>

Plot and copy solution figure after each task items.

1. Change maximum timestep from 1 to 200 days.
2. Change number of gridblocks from 30 to 300 and return timestep to 1.
3. Reduce rock heat capacity by 2 times and use resolution 300.
4. Increase rock thermal conductivity by 100 times and return heat capacity back.
